In [1]:
#!git clone https://github.com/whyhardt/SPICE.git

In [2]:
# !pip install -e SPICE

In [2]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt

from spice.estimator import SpiceEstimator
from spice.resources.spice_utils import SpiceConfig
from spice.utils.convert_dataset import convert_dataset
from spice.resources.rnn import BaseRNN

# For custom RNN
import torch
import torch.nn as nn

Let's load the data first with the `convert_dataset` method. This method returns a `SpiceDataset` object which we can use right away 

In [ ]:
import pandas as pd

df = pd.read_csv('../data/bustamante2023/bustamante2023.csv')
df['decision'] = df['decision'].replace({'stay': 0, 'exit': 1})
df['decision_duration'] = df['decision'].replace({'stay': 2, 'exit': 8.333})
df['harvest_duration'] = (df['choice_duration'] + df['harvest_duration'])/1000
df['travel_duration'] = df['travel_duration']/1000
df.to_csv('../data/bustamante2023/bustamante2023_processed.csv', index=False)

In [21]:
# Load your data
dataset = convert_dataset(
    file = '../data/bustamante2023/bustamante2023_processed.csv',
    df_participant_id='subject_id',
    df_choice='decision',
    df_reward='reward',
    df_block='overall_round',
    additional_inputs=['decision_duration'],
    timeshift_additional_inputs=False,
    )

# structure of dataset:
# dataset has two main attributes: xs -> inputs; ys -> targets (next action)
# shape: (n_participants*n_blocks*n_experiments, n_timesteps, features)
# features are (n_actions * action, n_actions * reward, n_additional_inputs * additional_input, block_number, experiment_id, participant_id)

# in order to set up the participant embedding we have to compute the number of unique participants in our data 
# to get the number of participants n_participants we do:
n_participants = len(dataset.xs[..., -1].unique())

print(f"Shape of dataset: {dataset.xs.shape}")
print(f"Number of participants: {n_participants}")
n_actions = dataset.ys.shape[-1]
print(f"Number of actions in dataset: {n_actions}")
print(f"Number of additional inputs: {dataset.xs.shape[-1]-2*n_actions-3}")

Shape of dataset: torch.Size([4296, 112, 8])
Number of participants: 537
Number of actions in dataset: 2
Number of additional inputs: 1


In [22]:
dataset.xs[0, :10, :]

tensor([[1.0000, 0.0000, 0.7286,    nan, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.6610,    nan, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000,    nan, 0.0492, 1.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.7242,    nan, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.6174,    nan, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.5456,    nan, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000,    nan, 0.0492, 1.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.8390,    nan, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.6964,    nan, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000,    nan, 0.0492, 1.0000, 0.0000, 0.0000, 0.0000]])

Now we are going to define the configuration for SPICE with a `SpiceConfig` object.

The `SpiceConfig` takes as arguments 
1. `library_setup (dict)`: Defining the variable names of each module.
2. `memory_state (dict)`: Defining the memory state variables and their initial values.
3. `states_in_logit (list)`: Defining which of the memory state variables are used later for the logit computation. This is necessary for some background processes.  

In [40]:
spice_config = SpiceConfig(
    library_setup={
        'value_stay': ['reward', 'decision_duration', 'value_stay'],
        'value_exit': ['reward', 'decision_duration', 'value_exit'],
    },
    
    memory_state={
            'value': 0.5,
        }
)

And now we are going to define the SPICE model which is a child of the `BaseRNN` and `torch.nn.Module` class and takes as required arguments:
1. `spice_config (SpiceConfig)`: previously defined SpiceConfig object
2. `n_actions (int)`: number of possible actions in your dataset (including non-displayed ones if applicable).
3. `n_participants (int)`: number of participants in your dataset.

As usual for a `torch.nn.Module` we have to define at least the `__init__` method and the `forward` method.
The `forward` method gets called when computing a forward pass through the model and takes as inputs `(inputs (SpiceDataset.xs), prev_state (dict, default: None), batch_first (bool, default: False))` and returns `(logits (torch.Tensor, shape: (n_participants*n_blocks*n_experiments, timesteps, n_actions)), updated_state (dict))`. Two necessary method calls inside the forward pass are:
1. `self.init_forward_pass(inputs, prev_state, batch_first) -> SpiceSignals`: returns a `SpiceSignals` object which carries all relevant information already processed.
2. `self.post_forward_pass(SpiceSignals, batch_first) -> SpiceSignals`: does some re-arranging of the logits to adhere to `batch_first`.

In [49]:
class SPICERNN(BaseRNN):
    
    def __init__(self, spice_config, n_participants, **kwargs):
        super().__init__(n_actions=2, spice_config=spice_config, n_participants=n_participants, embedding_size=32)
        
        # participant embedding
        self.participant_embedding = self.setup_embedding(num_embeddings=n_participants, embedding_size=self.embedding_size, dropout=0.)
        
        # scaling factor (inverse noise temperature) for each participant for the values which are handled by an hard-coded equation
        self.betas['value'] = self.setup_constant(embedding_size=self.embedding_size)
        
        # set up the submodules
        self.submodules_rnn['value_stay'] = self.setup_module(input_size=3+self.embedding_size)
        self.submodules_rnn['value_exit'] = self.setup_module(input_size=3+self.embedding_size)
        
    def forward(self, inputs, prev_state, batch_first=False):
        
        spice_signals = self.init_forward_pass(inputs, prev_state, batch_first)
        
        decision_duration = spice_signals.additional_inputs[..., 0].unsqueeze(-1).repeat(1, 1, self.n_actions)
        decision_duration = torch.where(decision_duration==0, 0.240, decision_duration)
        rewards_chosen = (spice_signals.actions * spice_signals.rewards.nan_to_num(0)).sum(dim=-1, keepdim=True).repeat(1, 1, self.n_actions)
        
        # time-invariant participant features
        participant_embeddings = self.participant_embedding(spice_signals.participant_ids)
        beta_reward = self.betas['value'](participant_embeddings)
        mask_stay = torch.tensor((1,0)).reshape(1, 1, self.n_actions).repeat(rewards_chosen.shape[0], rewards_chosen.shape[1], 1)
        mask_exit = torch.tensor((0,1)).reshape(1, 1, self.n_actions).repeat(rewards_chosen.shape[0], rewards_chosen.shape[1], 1)
        
        for timestep in spice_signals.timesteps:
            
            value_stay = self.state['value'][..., 0][:, None].repeat(1, 2)
            value_exit = self.state['value'][..., 1][:, None].repeat(1, 2)
            
            # update chosen value
            self.call_module(
                key_module='value_stay',
                key_state='value',
                action_mask=mask_stay[timestep],
                inputs=(rewards_chosen[timestep], decision_duration[timestep], value_exit),
                participant_index=spice_signals.participant_ids,
                participant_embedding=participant_embeddings,
                activation_rnn=torch.nn.functional.sigmoid,
            )
            
            # update not chosen value
            self.call_module(
                key_module='value_exit',
                key_state='value',
                action_mask=mask_exit[timestep],
                inputs=(rewards_chosen[timestep], decision_duration[timestep], value_stay),
                participant_index=spice_signals.participant_ids,
                participant_embedding=participant_embeddings,
                activation_rnn=torch.nn.functional.sigmoid,
            )
            
            # transform logits from item-space to action-space
            spice_signals.logits[timestep] = self.state['value'] * beta_reward
            
        spice_signals = self.post_forward_pass(spice_signals, batch_first)
        
        return spice_signals.logits, self.get_state()

Let's setup now the `SpiceEstimator` object and fit it to the data!

In [50]:
estimator = SpiceEstimator(
        # model paramaeters
        rnn_class=SPICERNN,
        spice_config=spice_config,
        n_actions=2,
        n_participants=n_participants,
        n_experiments=1,
        
        # training parameters
        epochs=10,  # 1000
        l2_weight_decay=0.01,
        sindy_epochs=10,  # 5000
        sindy_threshold=0.1,
        sindy_threshold_frequency=100,
        sindy_weight=0.01,
        sindy_library_polynomial_degree=2,
        verbose=True,
        save_path_spice='../params/bustamante2023/spice_bustamante2023.pkl',
    )

print(f"\nStarting training on {estimator.device}...")
print("=" * 80)
estimator.fit(dataset.xs, dataset.ys)
# estimator.load_spice(args.model)
print("=" * 80)
print("\nTraining complete!")

# Print example SPICE model for first participant
print("\nExample SPICE model (participant 0):")
print("-" * 80)
estimator.print_spice_model(participant_id=0)
print("-" * 80)


Starting training on cpu...

Training the RNN...
Epoch 10/10 --- L(Train): 0.6422949; Time: 3.61s; Convergence: 1.18e-02
Maximum number of training epochs reached.
Model did not converge yet.
Starting second stage SINDy fitting (threshold=0, single model)
SINDy Stage 2 - Epoch 10/10 --- L(Train): 0.0014354; Time: 0.18s
Second stage SINDy fitting complete!

Refitted SPICE model (participant 0):
--------------------------------------------------------------------------------
value_stay[t+1] = 0.006 1 + 0.0038 value_stay[t] + 0.0094 reward + 0.0046 decision_duration + 0.0065 value_stay[t] + 0.0036 value_stay*reward + 0.0024 value_stay*decision_duration + 0.0012 value_stay*value_stay + 0.0124 reward^2 + 0.0072 reward*decision_duration + 0.012 reward*value_stay + 0.0016 decision_duration^2 + 0.0055 decision_duration*value_stay + 0.0087 value_stay^2 
value_exit[t+1] = -0.0046 1 + -0.0048 value_exit[t] + -0.0018 reward + -0.0076 decision_duration + -0.0044 value_exit[t] + -0.0048 value_exit^